<a href="https://colab.research.google.com/github/mmistroni/OpenBB_Experiments/blob/main/OpenBBAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### What we will attempt to  do here is to create an Agent that will call Openbb functions and big query
### We will use Smolagents first and then we will expand little by little

# Importing Libraries

In [1]:
!pip install litellm

# Important!!!
#
# <---- Set your 'OPENAI_API_KEY' as a secret over there with the "key" icon
#
#
import os
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = api_key

In [2]:
# Install the OpenBB Platform with all available extensions.
# Messages indicating package version conflicts at the end of installation can be safely ignored.

!pip install openbb
!pip install ta
!pip install openbb-charting
# There is also a nightly distribution available, openbb-nightly

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=2d4baf89fb000e3e1558871d848dc2cf1582dc257da37ba3ae3ef7434b29563e
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta
  Using cached numba-0.61.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 47.5 MB/s eta 0:00:00
Using cached numba-0.61.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 11.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling s

In [3]:
!pip install smolagents -U
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

In [5]:
#!pip install --upgrade numpy
#!pip install --upgrade pandas
import requests
import pandas as pd
import numpy as np
from datetime import date, datetime
!mkdir -p data
!pip install google-api-python-client
from apiclient.http import MediaIoBaseDownload
!pip install fsspec
!pip install gcsfs
#!pip i#nstall pandas-datareader==0.10.0
#!pip install pandasai
#!pip install openai

#import pandas_datareader.data as dr
from pandas.tseries.offsets import BDay

import numpy as np
from datetime import datetime, date
from pandas.tseries.offsets import BDay
#import pandas_datareader.data as dr
#import scipy.stats as stats
from google.colab import auth
auth.authenticate_user()


print('Authenticated')

Authenticated


In [6]:
from google.cloud import storage
from googleapiclient.discovery import build
from apiclient.http import MediaIoBaseDownload
from datetime import date


def read_from_bucket(filename, bucket_name, to_panda=True):
  gcs_service = build('storage', 'v1')
  # The name for the new bucket
  holder = 'data/{}'.format(filename.split('/')[-1])
  print('Reading {} from {}'.format(filename,bucket_name))
  with open(holder, 'wb') as f:
    request = gcs_service.objects().get_media(bucket=bucket_name,
                                              object=filename)
    media = MediaIoBaseDownload(f, request)

    done = False
    while not done:
      # _ is a placeholder for a progress object that we ignore.
      # (Our file is small, so we skip reporting progress.)
      _, done = media.next_chunk()

  print('Download complete')
  if to_panda:
    return pd.read_csv(holder)


from google.cloud import bigquery
from google.cloud import bigquery_storage
# Construct a BigQuery client object.

def get_bigquery_as_dataframe(sql_stmnt):
  client = bigquery.Client(project='datascience-projects')
  bqstorageclient = bigquery_storage.BigQueryReadClient()
  query_job = client.query(sql_stmnt)
  return query_job.result().to_dataframe()

from google.cloud import bigquery
from google.cloud import bigquery_storage

def get_bq_as_pandas(sql):
  bqclient = bigquery.Client(project='datascience-projects')
  dataframe = (
    bqclient.query(sql)
    .result()
    .to_dataframe()
  )
  return dataframe


# Compare vs industry. Check metrics against every other company  until we can find industry / sector metrics





In [7]:
def read_bigquery_table(table_name):
  import bigframes.pandas as bf
  bf.options.bigquery.location = "US" #this variable is set based on the dataset you chose to query
  bf.options.bigquery.project = "datascience-projects" #this variable is set based on the dataset you chose to query
  return bf.read_gbq(f"datascience-projects.gcp_shareloader.{table_name}") #this variable is set based on the dataset you chose to query

### Smolagent tools

In [8]:
from smolagents import CodeAgent, tool

@tool
def suggest_menu(occasion: str) -> str:
    """
    Suggests a menu based on the occasion.
    Args:
        occasion (str): The type of occasion for the party. Allowed values are:
                        - "casual": Menu for casual party.
                        - "formal": Menu for formal party.
                        - "superhero": Menu for superhero party.
                        - "custom": Custom menu.
    """
    if occasion == "casual":
        return "Pizza, snacks, and drinks."
    elif occasion == "formal":
        return "3-course dinner with wine and dessert."
    elif occasion == "superhero":
        return "Buffet with high-energy and healthy food."
    else:
        return "Custom menu for the butler."

#open bb




In [9]:
agent = CodeAgent(tools=[suggest_menu], model=InferenceClientModel())

agent.run("Prepare a formal menu for the party.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a formal menu for the party.                                                                            │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = suggest_menu(occasion="formal")                                                                    
  print(formal_menu)                                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
3-course dinner with wine and dessert.

Out: None

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  formal_menu = {                                                                                                  
      "Appetizer": "Smoked Salmon with Cream Cheese and Cucumber",                                                 
      "Main Course": "Herb-Crusted Rack of Lamb with Roasted Vegetables",                                          
      "Dessert": "Chocolate Lava Cake with Vanilla Ice Cream",                                                     
      "Beverages": "Red and White Wine Selection"                                                                  
  }                                                                                                                
                                                                                                                   
  print(formal_menu)                                                                                               
  final_answer(formal_menu)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{'Appetizer': 'Smoked Salmon with Cream Cheese and Cucumber', 'Main Course': 'Herb-Crusted Rack of Lamb with 
Roasted Vegetables', 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream', 'Beverages': 'Red and White Wine 
Selection'}

Final answer: {'Appetizer': 'Smoked Salmon with Cream Cheese and Cucumber', 'Main Course': 'Herb-Crusted Rack of 
Lamb with Roasted Vegetables', 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream', 'Beverages': 'Red and White
Wine Selection'}

[Step 3: Duration 1.75 seconds| Input tokens: 6,470 | Output tokens: 230]

{'Appetizer': 'Smoked Salmon with Cream Cheese and Cucumber',
 'Main Course': 'Herb-Crusted Rack of Lamb with Roasted Vegetables',
 'Dessert': 'Chocolate Lava Cake with Vanilla Ice Cream',
 'Beverages': 'Red and White Wine Selection'}

In [ ]:
from litellm import completion
from typing import List, Dict


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="openai/gpt-4o",
        messages=messages,
        max_tokens=1024
    )
    return response.choices[0].message.content


what_to_help_with = input("What do you need help with?")

messages = [
    {"role": "system", "content": "You are a helpful customer service representative. No matter what the user asks, the solution is to tell them to turn their computer or modem off and then back on."},
    {"role": "user", "content": what_to_help_with}
]

response = generate_response(messages)
print(response)

What do you need help with?I need to get oranges 
It sounds like you might be experiencing some confusion. If you're having trouble accessing a grocery delivery service or finding a store online, I recommend turning your computer or modem off and then back on. This can often resolve connectivity issues and help you access the resources you need to find and purchase oranges.
